In [1]:
"""
Runs on CUDA, so make sure to have a GPU available
Takes 2 hours to run on an Nvidia 2080ti
"""
from dataloader import loadForumData

dir = "./data/hackerForums/"

df = loadForumData(dir)

# No clue why these Null values are here, should be filtered in the dataloader already
df = df[~df["PostContent"].isnull()]

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [4]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
from datasets import Dataset

tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")

ner = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True, device="cuda")

c:\Basis\dev\AU-Introduction-NLP\.venv\lib\site-packages\transformers\pipelines\token_classification.py:168: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


In [14]:
"""
Computation of the whole dataset takes a long time, so we split the dataset into chunks and save the results after each chunk.
We also print progress, time stats and estimated time left.
"""
output_file = "./data/HackerForumEntities.csv"

# Imagine spending 2 hours on some code that works ok, and then you want to add one little thing, just to find out there is a built-in function for the first thing you did that does everything you want and more
dataset = Dataset.from_pandas(df)
results = dataset.map(lambda row: {"entities": ner(row["PostContent"])}, batched=True)
results_df = results.to_pandas()
results_df.to_csv(output_file, index=False, mode="w", header=True)        

Map:   0%|          | 0/492342 [00:00<?, ? examples/s]